## GDELT

## Load MySQL

In [8]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = '3306'
username = 'root'
password = ''
database = 'gdelt'
localPath = 'D:/GDELT/'
outputPath = localPath + 'csv/'
tahunData = '2014'

csvHeader = ["GlobalEventId", "sqldate", "MonthYear","Year","FractionDate",
                               "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode",
                               "Actor1EthnicCode","Actor1Religion1Code","Actor1Religion2Code",
                               "Actor1Type1Code","Actor1Type2Code","Actor1Type3Code","Actor2Code",
                               "Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2EthnicCode",
                               "Actor2Religion1Code","Actor2Religion2Code","Actor2Type1Code","Actor2Type2Code",
                               "Actor2Type3Code","IsRootEvent","EventCode","EventBaseCode","EventRootCode",
                               "QuadClass","GoldsteinScale","NumMentions","NumSources","NumArticles",
                               "AvgTone","Actor1Geo_Type","Actor1Geo_FullName","Actor1Geo_CountryCode",
                               "Actor1Geo_ADM1Code","Actor1Geo_Lat","Actor1Geo_Long","Actor1Geo_FeatureID",
                               "Actor2Geo_Type","Actor2Geo_FullName","Actor2Geo_CountryCode","Actor2Geo_ADM1Code",
                               "Actor2Geo_Lat","Actor2Geo_Long","Actor2Geo_FeatureID","ActionGeo_Type",
                               "ActionGeo_FullName","ActionGeo_CountryCode","ActionGeo_ADM1Code",
                               "ActionGeo_Lat","ActionGeo_Long","ActionGeo_FeatureID","DATEADDED",
                               "SOURCEURL"]

csvHeaderNew = ["GlobalEventId", "sqldate", "MonthYear","Year",
               "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode","Actor1Type1Code",
               "Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2Type1Code",
               "IsRootEvent","EventBaseCode",
               "QuadClass","GoldsteinScale","NumMentions","NumSources","NumArticles","AvgTone",
               "Actor1Geo_FullName","Actor1Geo_CountryCode","Actor1Geo_ADM1Code","Actor1Geo_Lat","Actor1Geo_Long",
               "Actor2Geo_FullName","Actor2Geo_CountryCode","Actor2Geo_ADM1Code","Actor2Geo_Lat","Actor2Geo_Long",
               "ActionGeo_FullName","ActionGeo_CountryCode","ActionGeo_ADM1Code","ActionGeo_Lat","ActionGeo_Long",
               "SOURCEURL"]

# create connection to database
engine = create_engine('mysql+pymysql://' + username + ':' + password + '@' + host + ':' + port + '/' + database)

def run(sql):
    df = pd.read_sql_query(sql, engine)
    return df

## Load Folder Content 

In [12]:
import glob

compressedFiles = glob.glob(localPath + tahunData + "*.zip") 
print("Total Files : ", len(compressedFiles))

Total Files :  290


## Extract

In [45]:
import os.path
import urllib
import zipfile
import glob
import operator

print('extract ZIP File ke folder tmp')
for compressedFile in compressedFiles:
    # extract the contents of the compressed file to a temporary directory
    z = zipfile.ZipFile(file=compressedFile, mode='r')
    z.extractall(path=localPath + 'tmp/')
print('extract ZIP File Selesai')

extract ZIP File ke folder tmp
extract ZIP File Selesai


## Data Cleaning

In [28]:
# parse each of the csv files in the working directory,
print('Filter Content : Mulai')

csvFiles = glob.glob(localPath + 'tmp/' + tahunData + '*')
for inFileName in csvFiles:

    csvData = pd.read_csv(inFileName, delimiter = "\t", encoding = 'utf8', error_bad_lines=False, 
                       header=None, low_memory=False,
                       names = csvHeader)
    
    csvDataNew = csvData.loc[(csvData.EventBaseCode == 91) | (csvData.EventBaseCode == 112)]
    pd.DataFrame(csvDataNew).to_csv(outputPath + tahunData + '.csv', index=False, header=None, mode='a')
    
    # delete the temporary file

print('Filter Content: Selesai')

Filter Content : Mulai
Filter Content: Selesai


## Remove Unused Field

In [9]:
csvData = pd.read_csv(outputPath + tahunData + '.csv', delimiter = ",", encoding = 'utf8', error_bad_lines=False, 
                       header=None, low_memory=False,
                       names = csvHeader)

csvDataNew = csvData[csvHeaderNew]
csvDataNew.head(3)

pd.DataFrame(csvDataNew).to_csv(outputPath + tahunData + '-clean.csv', index=False, header=None, mode='w')

## Save to MySQL

In [ ]:
dfFromCSV = pd.read_csv(outputPath + tahunData + '.csv', sep = ",", header = None, names = csvHeader)

dfFromCSV.head(3)

dfFromCSV.to_sql(name = tahunData, con = engine, if_exists = 'append', index = False )

C:\Users\lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Scrap content from the url

In [10]:
from newspaper import Article

for row in df:
    try:
        print(row['SOURCEURL'])
        article = Article(row["SOURCEURL"])
        article.download()
        article.parse()
        print(article.title)
    except:
        pass

ModuleNotFoundError: No module named 'newspaper'

In [28]:
# Basic use and new schema method
import gdelt

gd= gdelt.gdelt()

events = gd.Search(['2017 May 23'],table='events',output='gpd',normcols=True,coverage=False)

# new schema method
print(gd.schema('events'))

ValueError: geopandas is installed but the geospatial functions failed for some other reason. Review the stack trace to see where the failure occurred.

In [12]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient()  # Remember your uri string
col = client['test']['test']

data = csvDataNew.to_dict(orient='records')  # Here's our added param..

col.insert_many(data)